In [1]:
import pandas as pd
import os
import sys
import numpy as np
import tensorflow as tf
import tensorflow.keras as krs
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import pickle

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from tqdm.autonotebook import tqdm
from tqdm import trange

In [2]:
tf.__version__

'2.0.0'

In [3]:
%matplotlib inline

In [4]:
%load_ext autoreload

%autoreload 2

In [5]:
if os.getcwd().endswith("notebooks"):
    os.chdir(os.pardir)

In [6]:
sys.path.append(os.getcwd())

In [15]:
from src.data.train_data_utils import prepare_experiment_data
from src.models.training_utils import generator_ratings, generator_ratings_features, mae_masked, mape_masked, mse_masked
from src.models.train_model import get_collabfiltering_model1, build_shallow_autorec_single_input, build_autorec_multi_input2, get_array

# Data prep

In [16]:
exd = prepare_experiment_data()

Processed: 0.0%
Processed: 0.27373261797875836%
Processed: 0.5474652359575167%
Processed: 0.8211978539362751%
Processed: 0.0%
Processed: 0.4555393586005831%
Processed: 0.9110787172011662%
Processed: 0.0%
Processed: 0.6804109682248077%


In [17]:
ytest = np.expand_dims(exd.Xraw_test.rating.to_numpy(), axis=1)

In [18]:
fpath = os.path.join(os.getcwd(), "data", "processed", "experiment")
with open(fpath, "wb") as ff:
    pickle.dump(exd, ff)

In [19]:
exd = pickle.load(open(fpath, "rb"))

In [22]:
exd.feature_names

['user_attr_Female',
 'user_attr_Male',
 'user_attr_missing',
 'model_attr_Female',
 'model_attr_Female&Male',
 'model_attr_Male',
 'brand_Apple',
 'brand_Archos',
 'brand_Bose',
 'brand_Canon',
 'brand_Cooper Cases',
 'brand_DBPOWER',
 'brand_DURAGADGET',
 'brand_EINCAR',
 'brand_EldHus',
 'brand_Etre Jeune',
 'brand_Fintie',
 'brand_Fujifilm',
 'brand_Funlux',
 'brand_Garmin',
 'brand_Gary Fong',
 'brand_Generic',
 'brand_HP',
 'brand_IRULU',
 'brand_JLAB',
 'brand_JVC',
 'brand_Jabra',
 'brand_Kensington',
 'brand_Kodak',
 'brand_Koolertron',
 'brand_LSS',
 'brand_Linksys',
 'brand_Logitech',
 'brand_Mpow',
 'brand_Neewer',
 'brand_Nikon',
 'brand_Olympus',
 'brand_Panasonic',
 'brand_Philips',
 'brand_Plemo',
 'brand_Polaroid',
 'brand_Pyle',
 'brand_Samsung',
 'brand_Savage',
 'brand_Sennheiser',
 'brand_Skullcandy',
 'brand_Sony',
 'brand_TaoTronics',
 'brand_Tiamat',
 'brand_Toshiba',
 'brand_Uniden',
 'brand_ViewSonic',
 'brand_Vivitar',
 'brand_XShields',
 'brand_ebasy',
 'bra

# Model prep

## Collaborative filtering with bias

In [23]:
dim_embed = 30
nusers = max(exd.Xraw_train.user_id.max(), exd.Xraw_test.user_id.max(), exd.Xraw_valid.user_id.max())
nitems = exd.Xraw_train.item_id.max()

user_input = krs.Input(shape=(1,), name="user_input", dtype='int32')
u = krs.layers.Embedding(nusers + 1, dim_embed)(user_input)
u = krs.layers.Flatten()(u)
u_bias = tf.keras.layers.Flatten()(krs.layers.Embedding(nusers + 1, 1)(user_input))

item_input = krs.Input(shape=(1,), name='item_input', dtype='int32')
i = krs.layers.Embedding(nitems + 1, dim_embed)(item_input)
i = krs.layers.Flatten()(i)
i_bias = krs.layers.Flatten()(krs.layers.Embedding(nitems + 1, 1)(item_input))

ui = krs.layers.Dot(axes=(1,1))([u, i])

all_features = krs.layers.Concatenate()([ui, u_bias, i_bias])
drop1 = krs.layers.Dropout(0.4)(all_features)
out = krs.layers.Dense(1)(drop1)

model_mf = krs.Model(inputs=[user_input, item_input], outputs=out)
model_mf.compile(loss=mse_masked, optimizer='adam', metrics=[mse_masked, 'mae', mape_masked])

In [24]:
model_mf.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 30)        610050      user_input[0][0]                 
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 30)        196680      item_input[0][0]                 
______________________________________________________________________________________________

In [25]:
x_mf = [
        exd.Xraw_train.user_id.to_numpy(),
        exd.Xraw_train.item_id.to_numpy()
    ]
y_mf = exd.Xraw_train.rating.to_numpy()

x_val_mf = [
    exd.Xraw_valid.user_id.to_numpy(),
    exd.Xraw_valid.item_id.to_numpy()
]
y_val_mf = exd.Xraw_valid.rating.to_numpy()



In [26]:
modelcf_history = model_mf.fit(x=x_mf, y=y_mf, batch_size=32, epochs=10, validation_data=(x_val_mf, y_val_mf))

Train on 36532 samples, validate on 14697 samples
Epoch 1/10
36532/36532 [==============================] - 17s 474us/sample - loss: 14.1435 - mse_masked: 14.1421 - mae: 3.5473 - mape_masked: 0.8137 - val_loss: 9.5894 - val_mse_masked: 9.5841 - val_mae: 2.8979 - val_mape_masked: 0.6496
Epoch 2/10
36532/36532 [==============================] - 15s 407us/sample - loss: 6.1027 - mse_masked: 6.1016 - mae: 2.2152 - mape_masked: 0.5339 - val_loss: 4.3424 - val_mse_masked: 4.3398 - val_mae: 1.9098 - val_mape_masked: 0.4772
Epoch 3/10
36532/36532 [==============================] - 16s 450us/sample - loss: 2.6907 - mse_masked: 2.6907 - mae: 1.3903 - mape_masked: 0.4006 - val_loss: 2.8652 - val_mse_masked: 2.8639 - val_mae: 1.5554 - val_mape_masked: 0.4296
Epoch 4/10
36532/36532 [==============================] - 16s 449us/sample - loss: 1.6199 - mse_masked: 1.6197 - mae: 1.0458 - mape_masked: 0.3352 - val_loss: 2.4179 - val_mse_masked: 2.4173 - val_mae: 1.4329 - val_mape_masked: 0.4155
Epoch 5/

In [27]:
yhat_mf = model_mf.predict(x=[
    exd.Xraw_test.user_id.to_numpy(dtype='int32'),
    exd.Xraw_test.item_id.to_numpy(dtype='int32')
])

In [28]:
mape_masked(ytest, yhat_mf).numpy()

0.39153633

## Deep recommender

In [29]:
from sklearn.preprocessing import StandardScaler

ytrain_mu = np.mean(exd.Xraw_train.rating)
ytrain_norm = (exd.Xraw_train.rating - ytrain_mu).to_numpy()
yval_norm = (exd.Xraw_valid.rating - ytrain_mu).to_numpy()
ytest_norm = (exd.Xraw_test.rating - ytrain_mu).to_numpy()

In [30]:
exd.Xfeatures_train.shape

(36532, 87)

In [99]:
dim_embed = 30
nusers = max(exd.Xraw_train.user_id.max(), exd.Xraw_test.user_id.max(), exd.Xraw_valid.user_id.max())
nitems = exd.Xraw_train.item_id.max()

f_input = krs.Input(shape=(exd.Xfeatures_train.shape[1],), name="features_input", dtype='float32')
f_drop = krs.layers.Dropout(rate=0.3)(f_input)
f_dense = krs.layers.Dense(16, activation='relu', kernel_initializer=krs.initializers.GlorotNormal())(f_drop) #, kernel_regularizer=krs.regularizers.l1_l2(l1=0.001, l2=0.001))(f_input)
#f_drop2 = krs.layers.Dropout(rate=0.3)(f_dense)
#f_dense2 = krs.layers.Dense(16, activation="relu", kernel_initializer=krs.initializers.GlorotNormal())(f_drop2)


user_input = krs.Input(shape=(1,), name="user_input", dtype='int32')
u = krs.layers.Embedding(nusers + 1, dim_embed)(user_input)
u = krs.layers.Flatten()(u)
u_bias = tf.keras.layers.Flatten()(krs.layers.Embedding(nusers + 1, 1)(user_input))

item_input = krs.Input(shape=(1,), name='item_input', dtype='int32')
i = krs.layers.Embedding(nitems + 1, dim_embed)(item_input)
i = krs.layers.Flatten()(i)
i_bias = krs.layers.Flatten()(krs.layers.Embedding(nitems + 1, 1)(item_input))

#ui = krs.layers.Dot(axes=(1,1))([u, i])
ui = krs.layers.Multiply()([u, i])

all_features = krs.layers.Concatenate(axis=1)([ui, f_dense, u_bias, i_bias])
drop =  krs.layers.Dropout(rate=0.2)(all_features)
d1 = krs.layers.Dense(32, activation="relu", kernel_initializer=krs.initializers.GlorotNormal())(drop) #,  kernel_regularizer=krs.regularizers.l1_l2(l1=0.001, l2=0.001))(all_features)
drop2 = krs.layers.Dropout(rate=0.2)(d1)
d2 = krs.layers.Dense(16, activation="relu", kernel_initializer=krs.initializers.GlorotNormal())(drop2) #,  kernel_regularizer=krs.regularizers.l1_l2(l1=0.001, l2=0.001))(d1)
out = krs.layers.Dense(1, )(d2)

model = krs.Model(inputs=[user_input, item_input, f_input], outputs=out )
model.compile(loss=mse_masked, optimizer='adam', metrics=[mse_masked, 'mae', mape_masked])

In [100]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 30)        610050      user_input[0][0]                 
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 30)        196680      item_input[0][0]                 
______________________________________________________________________________________________

In [101]:
x = [
        exd.Xraw_train.user_id.to_numpy(),
        exd.Xraw_train.item_id.to_numpy(),
        exd.Xfeatures_train
    ]
y = exd.Xraw_train.rating.to_numpy()

x_val = [
    exd.Xraw_valid.user_id.to_numpy(),
    exd.Xraw_valid.item_id.to_numpy(),
    exd.Xfeatures_valid
]
y_val = exd.Xraw_valid.rating.to_numpy()

In [102]:
es = krs.callbacks.EarlyStopping(monitor='val_mape_masked', mode='min', min_delta=0.005, patience=3)
model_history = model.fit(x=x, y=y, batch_size=16, epochs=10, validation_data=(x_val, y_val), callbacks=[es])

Train on 36532 samples, validate on 14697 samples
Epoch 1/10
36532/36532 [==============================] - 31s 857us/sample - loss: 2.0114 - mse_masked: 2.0118 - mae: 1.0874 - mape_masked: 0.4213 - val_loss: 1.3441 - val_mse_masked: 1.3449 - val_mae: 0.9188 - val_mape_masked: 0.3775
Epoch 2/10
36532/36532 [==============================] - 32s 866us/sample - loss: 1.1052 - mse_masked: 1.1050 - mae: 0.7961 - mape_masked: 0.3337 - val_loss: 1.4217 - val_mse_masked: 1.4224 - val_mae: 0.9900 - val_mape_masked: 0.3777
Epoch 3/10
36532/36532 [==============================] - 32s 868us/sample - loss: 0.4739 - mse_masked: 0.4738 - mae: 0.5059 - mape_masked: 0.2025 - val_loss: 1.3864 - val_mse_masked: 1.3872 - val_mae: 0.9108 - val_mape_masked: 0.3815
Epoch 4/10
36532/36532 [==============================] - 34s 918us/sample - loss: 0.2322 - mse_masked: 0.2322 - mae: 0.3423 - mape_masked: 0.1298 - val_loss: 1.3908 - val_mse_masked: 1.3916 - val_mae: 0.9318 - val_mape_masked: 0.3799


In [103]:
yhat = model.predict(x=[
    exd.Xraw_test.user_id.to_numpy(dtype='int32'),
    exd.Xraw_test.item_id.to_numpy(dtype='int32'),
    exd.Xfeatures_test.astype('float32')
])
#yhat_denorm = yhat + ytrain_mu

In [104]:
mape_masked(ytest, yhat).numpy() #yhat_denorm).numpy()

0.3913579

In [105]:
import gc
tf.keras.backend.clear_session()
gc.collect()

34624